In [1]:
import json
import hashlib
import os

import spacy

In [2]:
text = """
Bastien was born in New York.
Robert work as a Data Scientist.
Charles is the son of Robert. He was born in 1993. In 1993 Bastien went to the world in New York.
Charles graduated from EPFL in 2019. """

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
def split_by_sentence(text):
    doc = nlp(text)
    return [str(sent) for sent in doc.sents]
sents = split_by_sentence(text)

In [5]:
def encode_hash(obj):
    return hashlib.sha224(str(obj).encode()).hexdigest()

In [6]:
id2ent = {}
def detect_entities(sent):
    json_obj = []
    doc = nlp(sent)
    for ent in doc.ents:
        for ent2 in doc.ents:
            if ent != ent2:
                id2ent[encode_hash(ent)] = str(ent)
                json_obj.append({'sentence': sent,
                                'head': {
                                    'word': str(ent),
                                    'id': str(encode_hash(ent))
                                } ,
                                 'tail': {
                                     'word': str(ent2),
                                     'id': str(encode_hash(ent2))
                                 },
                                'relation': 'NA'})
    return json_obj

In [7]:
json_obj = []
for sent in sents:
    json_obj += detect_entities(sent)

In [10]:
with open('id2ent.json', 'w') as f:
    f.write(json.dumps(id2ent))

In [9]:
directory = '../data/pred'
if not os.path.exists(directory):
    os.mkdir(directory)
with open('../data/pred/test.json', 'w') as f:
    f.write(json.dumps(json_obj))